In [1]:
import numpy as np

## Lendo os arquivos

In [14]:
data_teste = np.loadtxt("teste.txt")
data_treino = np.loadtxt("treino.txt")

# Pegando dados de treino e normalizando utilizando Z-Score

In [47]:
X_teste = data_teste[:, :-1]  # Recursos teste
y_teste = data_teste[:, -1]   # Rótulos teste

# Calcule a média e o desvio padrão das características no conjunto de teste
mean_teste = X_teste.mean(axis=0)
std_teste = X_teste.std(axis=0)

# Encontre as características com variância zero no conjunto de teste
zero_variance_features = np.where(std_teste == 0)[0]

# Remova essas características dos conjuntos de teste
X_teste = np.delete(X_teste, zero_variance_features, axis=1)

# Calcule a média e o desvio padrão novamente após remover as características com variância zero
mean_teste = X_teste.mean(axis=0)
std_teste = X_teste.std(axis=0)

# Normalize o conjunto de teste usando o Z-score
X_test_normalized = (X_teste - mean_teste) / std_teste


# Pegando dados de teste e normalizando utilizando Z-Score


In [48]:
X_treino = data_treino[:, :-1] # Recursos treino
y_treino = data_treino[:, -1] # Rótulos treino

# Calcule a média e o desvio padrão das características no conjunto de treino
mean_treino = X_treino.mean(axis=0)
std_treino = X_treino.std(axis=0)

# Encontre as características com variância zero no conjunto de teste
zero_variance_features = np.where(std_treino == 0)[0]

# Remova essas características dos conjuntos de teste
X_treino = np.delete(X_treino, zero_variance_features, axis=1)

# Calcule a média e o desvio padrão novamente após remover as características com variância zero
mean_treino = X_treino.mean(axis=0)
std_treino = X_treino.std(axis=0)

# Normalize o conjunto de treinamento
X_train_normalized = (X_treino - mean_treino) / std_treino

### Importando bibliotecas para análise dos dados



In [49]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import accuracy_score

# Classe KNN

In [50]:
class KNN(object):
    def __init__(self, k):
        self.k = k
        self.pontos = None
        self.rotulos = None

    def fit(self, X, y):
        self.pontos = X
        self.rotulos = y

    # Distância Euclidiana
    @staticmethod
    def dist(p,q):
        return np.sqrt(np.sum((p-q)**2))

    def predict(self, X):
        predicoes = []
        # para cada instância p de X
        for p in X:
            # calcular a distância de p para todos os pontos do conjunto de treinamento
            #d = [self.dist(p,q) for q in self.pontos]
            d = pairwise_distances(self.pontos, p.reshape((1,-1))).flatten()
            # retornar os índices em ordem crescente de distância
            ds = np.argsort(d)
            # pegar apenas os índices dos k-vizinhos mais próximos
            knn = ds[:self.k]
            # pegar os rótulos dos k-vizinhos mais próximos
            rotulos = self.rotulos[knn]
            # contar os rótulos dos vizinhos mais próximos
            rotulos, cont = np.unique(rotulos, return_counts=True)
            # a predição do ponto p é o rótulo majoritário
            pred = rotulos[np.argmax(cont)]
            predicoes.append(pred)
        return np.array(predicoes)

### Valores de K para análise dos resultados


In [51]:
k_values = [1,3,5,7,9,11,13,15,17,19]

In [52]:
for k in k_values:
  knn = KNN(k=k)
  knn.fit(X_train_normalized, y_treino)
  pred = knn.predict(X_test_normalized)
  accuracy = accuracy_score(y_teste, pred)
  print(f"K = {k}, Acurácia: {accuracy}")

K = 1, Acurácia: 0.91
K = 3, Acurácia: 0.905
K = 5, Acurácia: 0.917
K = 7, Acurácia: 0.924
K = 9, Acurácia: 0.928
K = 11, Acurácia: 0.931
K = 13, Acurácia: 0.931
K = 15, Acurácia: 0.928
K = 17, Acurácia: 0.926
K = 19, Acurácia: 0.924


## Separando os conjuntos aleatóriamente em 25%, 50% e 100% dos dados de treinamento.

In [53]:
from sklearn.model_selection import train_test_split

In [58]:
# Dividir o conjunto de treinamento em diferentes proporções
X_train_25, X_unused, y_train_25, y_unused = train_test_split(X_train_normalized, y_treino, test_size=0.75, random_state=42)
X_train_50, X_unused, y_train_50, y_unused = train_test_split(X_train_normalized, y_treino, test_size=0.5, random_state=42)
X_train_100 = X_train_normalized
y_train_100 = y_treino

# Treinar e avaliar o modelo KNN com diferentes divisões
for k in k_values:
  print(f"K = {k}")
  for X_train, y_train, label in [(X_train_25, y_train_25, '25%'), (X_train_50, y_train_50, '50%'), (X_train_100, y_train_100, '100%')]:
      knn = KNN(k=k)
      knn.fit(X_train, y_train)
      pred = knn.predict(X_test_normalized)
      accuracy = accuracy_score(y_teste, pred)
      print(f"Divisão: {label}, Acurácia: {accuracy}")

K = 1
Divisão: 25%, Acurácia: 0.891
Divisão: 50%, Acurácia: 0.897
Divisão: 100%, Acurácia: 0.894
K = 3
Divisão: 25%, Acurácia: 0.907
Divisão: 50%, Acurácia: 0.912
Divisão: 100%, Acurácia: 0.904
K = 5
Divisão: 25%, Acurácia: 0.906
Divisão: 50%, Acurácia: 0.927
Divisão: 100%, Acurácia: 0.905
K = 7
Divisão: 25%, Acurácia: 0.894
Divisão: 50%, Acurácia: 0.922
Divisão: 100%, Acurácia: 0.93
K = 9
Divisão: 25%, Acurácia: 0.884
Divisão: 50%, Acurácia: 0.92
Divisão: 100%, Acurácia: 0.93
K = 11
Divisão: 25%, Acurácia: 0.876
Divisão: 50%, Acurácia: 0.91
Divisão: 100%, Acurácia: 0.927
K = 13
Divisão: 25%, Acurácia: 0.867
Divisão: 50%, Acurácia: 0.912
Divisão: 100%, Acurácia: 0.918
K = 15
Divisão: 25%, Acurácia: 0.853
Divisão: 50%, Acurácia: 0.906
Divisão: 100%, Acurácia: 0.921
K = 17
Divisão: 25%, Acurácia: 0.828
Divisão: 50%, Acurácia: 0.905
Divisão: 100%, Acurácia: 0.92
K = 19
Divisão: 25%, Acurácia: 0.82
Divisão: 50%, Acurácia: 0.899
Divisão: 100%, Acurácia: 0.918
